In [1]:
import re
from datasets import load_dataset, concatenate_datasets
from zemberek import TurkishSentenceExtractor

In [2]:
tquad = load_dataset('../../taboo-datasets/tquad2', split='train[:1%]')
tquad

2023-03-18 01:01:11,440 - datasets.builder - WARNING
Msg: Found cached dataset tquad2 (/Users/quimba/.cache/huggingface/datasets/tquad2/tquad2/2.0.0/ee17d1274749a6ce551c3c53fd89e7a3e58c4511d454f939628b2bad6b861091)



Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 142
})

In [126]:
xquad = load_dataset('xquad', 'xquad.tr', split='validation')

xquad = xquad.map(lambda x: {'context': x['context'].encode('iso-8859-9', 'ignore').decode('iso-8859-9')})
xquad

2023-03-02 21:52:43,775 - datasets.builder - WARNING
Msg: Found cached dataset xquad (/Users/quimba/.cache/huggingface/datasets/xquad/xquad.tr/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)

2023-03-02 21:52:43,779 - datasets.arrow_dataset - WARNING
Msg: Loading cached processed dataset at /Users/quimba/.cache/huggingface/datasets/xquad/xquad.tr/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336/cache-a6fc9a4262e90365.arrow



Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})

In [138]:
my_quad = concatenate_datasets([xquad, tquad]).remove_columns(['title'])
my_quad

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1332
})

In [3]:
my_quad=tquad

In [5]:
def get_clozes(x):
    extractor = TurkishSentenceExtractor()
    unique_answers = set()

    id_ = x['id']
    context = x['context']
    question = x['question']
    answer = x['answers']
    answer_texts, answer_starts = answer['text'], answer['answer_start']

    new_rows = {'id': [], 'context': [], 'question': [], 'answer': [], 'answer_start': [], 'cloze': []}
    for text, start in zip(answer_texts, answer_starts):
        for span in extractor.extract_to_spans(context):
            if text not in unique_answers and span.in_span(start):
                unique_answers.add(text)

                new_rows['id'].append(id_)
                new_rows['context'].append(context)
                new_rows['question'].append(question)
                new_rows['answer'].append(text)
                new_rows['answer_start'].append(start)
                new_rows['cloze'].append(span.get_sub_string(context))

    return new_rows


#my_quad = concatenate_datasets([xquad, tquad]).remove_columns(['title'])
my_quad = my_quad.map(get_clozes, remove_columns=['id', 'context', 'question', 'answers'])
my_quad = my_quad.map(lambda x: {'id': x['id'][0], 'context': x['context'][0], 'question': x['question'][0], 'answer': x['answer'][0], 'answer_start': x['answer_start'][0], 'cloze': x['cloze'][0]})

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [6]:
my_quad[0]

{'id': '56dde1d966d3e219004dad8d',
 'title': 'Normans',
 'context': "Rollo'nun gelişinden önce popülasyonları Picardy'den veya 'Franklar' olarak nitelendirilen Île-de-France'den farklı değildi. Daha önceki Viking yerleşimcileri 880'lerde gelmeye başlamıştı ancak doğudaki koloniler (Roumois ve Pays de Caux) arasında alçak Seine vadisi çevresinde ve batıdaki Cotentin Yarımadası'ndaki koloniler arasında bölündü ve hemen hemen hiçbir yabancı yerleşimci olmadan nüfusun aynı kaldığı geleneksel pagii ile ayrıldılar.  Normandiya ve Atlantik kıyılarının bir kısmına baskın düzenleyen ve yerleşen Rollo'nun muhalifleri Norveçliler, İskandinav-İskoçyalılar, Orkney Vikingler, muhtemelen İsveçliler ve Norveç kontrolündeki İngiliz Danelaw'lardan Anglo Danimarkalıları içermektedir. ",
 'question': 'Kim geldiğinde orijinal viking yerleşimcilerine ortak bir kimlik vermiştir?',
 'answer': 'Rollo',
 'answer_start': 0,
 'cloze': "Rollo'nun gelişinden önce popülasyonları Picardy'den veya 'Franklar' olarak ni

In [7]:
my_quad.to_csv('tquad2_clozes.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

148050